In [0]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import time

In [0]:
#Reading data
data = input_data.read_data_sets('data/fashion',one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/fashion/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/fashion/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting data/fashion/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting data/fashion/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/datas

In [0]:
# Shapes of training set
print("Training set (images) shape: {shape}".format(shape=data.train.images.shape))
print("Training set (labels) shape: {shape}".format(shape=data.train.labels.shape))

# Shapes of test set
print("Test set (images) shape: {shape}".format(shape=data.test.images.shape))
print("Test set (labels) shape: {shape}".format(shape=data.test.labels.shape))

Training set (images) shape: (55000, 784)
Training set (labels) shape: (55000, 10)
Test set (images) shape: (10000, 784)
Test set (labels) shape: (10000, 10)


In [0]:
# Reshape training and testing image
train_X = data.train.images.reshape(-1, 28, 28, 1)
test_X = data.test.images.reshape(-1,28,28,1)
train_y = data.train.labels
test_y = data.test.labels

###Our Model:
<ol>
<li>Convolution layer with 32-3x3 filters</li>
<li>Max pooling layer 2x2</li>
<li>Convolution layer with 64-3x3 filters</li>
<li>Max pooling layer 2x2</li>
<li>Convolution layer with 128-3x3 filters</li>
<li>Max pooling layer 2x2</li>
<li>Fully-connected layer of 128 units</li>
<li>Output layer of 10 units</li>
</ol>

Architecture drawn using: http://alexlenail.me/NN-SVG/AlexNet.html

<img src="https://i.imgur.com/DGGlRcK.png" alt="neural-network" border="0">

In [0]:
iters = 15    
learning_rate = 0.001   
batch_size = 256        #Mini-batch size
n_input = 28            #img_shape: 28x28
n_classes = 10          #no_classes: 10

In [0]:
#Define layers (wrappers) and initializer

def conv2d(x, W, b, strides=1):
    # implement the conv2d function to make a convolution layer and applies relu 
    # as activation function 

def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],padding='SAME')
  
  
def initializer():  
  weights = {
      'wc1': tf.get_variable('W0', shape=(3,3,1,32), initializer=tf.contrib.layers.xavier_initializer()), 
      # complete the missing weights for the second layer 
      'wc3': tf.get_variable('W2', shape=(3,3,64,128), initializer=tf.contrib.layers.xavier_initializer()), 
      'wd1': tf.get_variable('W3', shape=(4*4*128,128), initializer=tf.contrib.layers.xavier_initializer()), 
      'out': tf.get_variable('W6', shape=(128,n_classes), initializer=tf.contrib.layers.xavier_initializer()), 
  }
  biases = {
      'bc1': tf.get_variable('B0', shape=(32), initializer=tf.contrib.layers.xavier_initializer()),
      'bc2': tf.get_variable('B1', shape=(64), initializer=tf.contrib.layers.xavier_initializer()),
      'bc3': tf.get_variable('B2', shape=(128), initializer=tf.contrib.layers.xavier_initializer()),
      'bd1': tf.get_variable('B3', shape=(128), initializer=tf.contrib.layers.xavier_initializer()),
      'out': tf.get_variable('B4', shape=(10), initializer=tf.contrib.layers.xavier_initializer()),
  }
  return weights, biases

In [0]:
#Model

def conv_net(x, weights, biases):  
    
    # Convolution layers
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)
    conv3 = conv2d(conv2, weights['wc3'], biases['bc3'])
    conv3 = maxpool2d(conv3, k=2)


    # Fully connected layer
    fc1 = tf.reshape(conv3, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    
    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    
    return out

In [0]:
tf.reset_default_graph()

x = tf.placeholder("float", [None, 28,28,1])
y = tf.placeholder("float", [None, n_classes])

weights, biases = initializer()
# Applying the model to our data
pred = conv_net(x, weights, biases)
# Defining the cost function
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
# Defining the optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [0]:
#Here you check whether the index of the maximum value of the predicted image is equal to the actual labelled image. and both will be a column vector.
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
#calculate accuracy across all the given images and average them out. 
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [0]:
# Running session

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init) 
    train_loss = []
    test_loss = []
    train_accuracy = []
    test_accuracy = []
    #Write to graph
    summary_writer = tf.summary.FileWriter('./Output', sess.graph)
    for i in range(iters):
        start = time.time()
        for batch in range(len(train_X)//batch_size):
            batch_x = train_X[batch*batch_size:min((batch+1)*batch_size,len(train_X))]
            batch_y = train_y[batch*batch_size:min((batch+1)*batch_size,len(train_y))]    
            # Calculate batch loss and accuracy
            opt = sess.run(optimizer, feed_dict={x: batch_x,
                                                              y: batch_y})
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                              y: batch_y})
        print("(" + str(i+1) + "/" + str(iters) + ")" ", Loss= " + \
                      "{:.6f}".format(loss) + ", Training Accuracy= " + \
                      "{:.5f}".format(acc))
        end = time.time()

        # Calculate accuracy for all test images
        test_acc,valid_loss = sess.run([accuracy,cost], feed_dict={x: test_X,y : test_y})
        train_loss.append(loss)
        test_loss.append(valid_loss)
        train_accuracy.append(acc)
        test_accuracy.append(test_acc)
        print("Testing Accuracy:","{:.5f}".format(test_acc))
        print("Time taken for one epoch: " + str(end-start) + " seconds")
        print("")
    summary_writer.close()

(1/15), Loss= 0.196493, Training Accuracy= 0.96094
Testing Accuracy: 0.96470
Time taken for one epoch: 7.713947296142578 seconds

(2/15), Loss= 0.134302, Training Accuracy= 0.96875
Testing Accuracy: 0.97810
Time taken for one epoch: 5.12232780456543 seconds

(3/15), Loss= 0.092068, Training Accuracy= 0.97656
Testing Accuracy: 0.98400
Time taken for one epoch: 5.097223997116089 seconds

(4/15), Loss= 0.072345, Training Accuracy= 0.98047
Testing Accuracy: 0.98720
Time taken for one epoch: 4.797422409057617 seconds

(5/15), Loss= 0.057555, Training Accuracy= 0.98047
Testing Accuracy: 0.98840
Time taken for one epoch: 4.846061706542969 seconds

(6/15), Loss= 0.034990, Training Accuracy= 0.98047
Testing Accuracy: 0.98890
Time taken for one epoch: 4.920878648757935 seconds

(7/15), Loss= 0.022111, Training Accuracy= 0.99219
Testing Accuracy: 0.98750
Time taken for one epoch: 4.893008470535278 seconds

(8/15), Loss= 0.012125, Training Accuracy= 0.99609
Testing Accuracy: 0.98720
Time taken for

Connecting to TPU: https://stackoverflow.com/questions/52530578/how-use-tpu-in-google-colab

### ResNet 50

<h3>Deep Networks Problem (Vanishing Gradients) </h3>
<p> because of multiplication during backpropagation</p>
</br>
<img src="https://github.com/priya-dwivedi/Deep-Learning/raw/582d6c154cb8b3a0ea39ab77012e9bed0c831473/resnet_keras/images/vanishing_grad_kiank.png" alt="neural-network" border="0">

Solution: ResNet blocks with the shortcut makes it very easy for one of the blocks to learn an identity function
<h3>Difference between shortcut and no shortcut </h3>
</br>
<img src="https://cdn-images-1.medium.com/max/800/1*-_ED04HNCNz7HFyqQtbTtg.png" alt="neural-network" border="0">
<h3>ResNet architecture</h3>
</br>
<img src="https://github.com/priya-dwivedi/Deep-Learning/raw/582d6c154cb8b3a0ea39ab77012e9bed0c831473/resnet_keras/images/resnet_kiank.png" alt="neural-network" border="0">
<h3>Difference between ResNet and ResNeXt </h3>
</br>
<img src="https://cdn-images-1.medium.com/max/800/1*LOoc11tkDoqv0pC6OH7mwA.png" alt="neural-network" border="0">

In [0]:

from tensorflow.keras import layers
from tensorflow.keras import models


#
# image dimensions
#

img_height = 224
img_width = 224
img_channels = 3

#
# network params
#

cardinality = 1


def residual_network(x):
    """
    ResNeXt by default. For ResNet set `cardinality` = 1 above.
    
    """
    def add_common_layers(y):
        y = layers.BatchNormalization()(y)
        y = layers.LeakyReLU()(y)

        return y

    def grouped_convolution(y, nb_channels, _strides):
        # when `cardinality` == 1 this is just a standard convolution
        if cardinality == 1:
            return layers.Conv2D(nb_channels, kernel_size=(3, 3), strides=_strides, padding='same')(y)
        
#         assert not nb_channels % cardinality
#         _d = nb_channels // cardinality

#         # in a grouped convolution layer, input and output channels are divided into `cardinality` groups,
#         # and convolutions are separately performed within each group
#         groups = []
#         for j in range(cardinality):
#             group = layers.Lambda(lambda z: z[:, :, :, j * _d:j * _d + _d])(y)
#             groups.append(layers.Conv2D(_d, kernel_size=(3, 3), strides=_strides, padding='same')(group))
            
#         # the grouped convolutional layer concatenates them as the outputs of the layer
#         y = layers.concatenate(groups)

#         return y

    def residual_block(y, nb_channels_in, nb_channels_out, _strides=(1, 1), _project_shortcut=False):
        """
        Our network consists of a stack of residual blocks. These blocks have the same topology,
        and are subject to two simple rules:
        - If producing spatial maps of the same size, the blocks share the same hyper-parameters (width and filter sizes).
        - Each time the spatial map is down-sampled by a factor of 2, the width of the blocks is multiplied by a factor of 2.
        """
        shortcut = y

        # we modify the residual building block as a bottleneck design to make the network more economical
        y = layers.Conv2D(nb_channels_in, kernel_size=(1, 1), strides=(1, 1), padding='same')(y)
        y = add_common_layers(y)

        # ResNeXt (identical to ResNet when `cardinality` == 1)
        y = grouped_convolution(y, nb_channels_in, _strides=_strides)
        y = add_common_layers(y)

        y = layers.Conv2D(nb_channels_out, kernel_size=(1, 1), strides=(1, 1), padding='same')(y)
        # batch normalization is employed after aggregating the transformations and before adding to the shortcut
        y = layers.BatchNormalization()(y)

        # identity shortcuts used directly when the input and output are of the same dimensions
        if _project_shortcut or _strides != (1, 1):
            # when the dimensions increase projection shortcut is used to match dimensions (done by 1×1 convolutions)
            # when the shortcuts go across feature maps of two sizes, they are performed with a stride of 2
            shortcut = layers.Conv2D(nb_channels_out, kernel_size=(1, 1), strides=_strides, padding='same')(shortcut)
            shortcut = layers.BatchNormalization()(shortcut)

        y = layers.add([shortcut, y])

        # relu is performed right after each batch normalization,
        # expect for the output of the block where relu is performed after the adding to the shortcut
        y = layers.LeakyReLU()(y)

        return y

    # conv1
    x = layers.Conv2D(64, kernel_size=(7, 7), strides=(2, 2), padding='same')(x)
    x = add_common_layers(x)

    # conv2
    x = layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
    for i in range(3):
        project_shortcut = True if i == 0 else False
        x = residual_block(x, 128, 256, _project_shortcut=project_shortcut)

    # conv3
    for i in range(4):
        # down-sampling is performed by conv3_1, conv4_1, and conv5_1 with a stride of 2
        strides = (2, 2) if i == 0 else (1, 1)
        x = residual_block(x, 256, 512, _strides=strides)

    # conv4
    for i in range(6):
        strides = (2, 2) if i == 0 else (1, 1)
        x = residual_block(x, 512, 1024, _strides=strides)

    # conv5
    for i in range(3):
        strides = (2, 2) if i == 0 else (1, 1)
        x = residual_block(x, 1024, 2048, _strides=strides)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(1)(x)

    return x


image_tensor = layers.Input(shape=(img_height, img_width, img_channels,))
network_output = residual_network(image_tensor)
  
model = models.Model(inputs=[image_tensor], outputs=[network_output])
print(model.summary())

### Inception v3

<h3>WHY USE 1 LETS USE ALL</h3>
In a typical CNN layer, we make a choice to either have a 3x3 filters, or 5x5 filters or a max pooling layer. In general all of these are beneficial . The inception block use them all.
<h3>Efficiency</h3>
Use (1x1) Conv Layer to reduce dimensionality, then perform a regular conv layer, finally use another (1x1) to increase dimensionality (not used in inception).
</br>
<img src="https://cdn-images-1.medium.com/max/1600/1*aq4tcBl9t5Z36kTDeZSOHA.png" alt="neural-network" border="0">

In [0]:
import tensorflow.keras as keras
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, concatenate, Flatten, Dense
input_img = Input(shape = (32, 32, 3))

tower_1 = Conv2D(64, (1,1), padding='same', activation='relu')(input_img)
tower_1 = Conv2D(64, (3,3), padding='same', activation='relu')(tower_1)
tower_2 = Conv2D(64, (1,1), padding='same', activation='relu')(input_img)
tower_2 = Conv2D(64, (5,5), padding='same', activation='relu')(tower_2)
tower_3 = MaxPooling2D((3,3), strides=(1,1), padding='same')(input_img)
tower_3 = Conv2D(64, (1,1), padding='same', activation='relu')(tower_3)

output = concatenate([tower_1, tower_2, tower_3], axis = 3)

output = Flatten()(output)
out    = Dense(1, activation='sigmoid')(output)

model = keras.Model(inputs = input_img, outputs = out)
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1166 (Conv2D)            (None, 32, 32, 64)   256         input_7[0][0]                    
__________________________________________________________________________________________________
conv2d_1168 (Conv2D)            (None, 32, 32, 64)   256         input_7[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_6 (MaxPooling2D)  (None, 32, 32, 3)    0           input_7[0][0]                    
__________________________________________________________________________________________________
conv2d_116